# Active Learning Spectrally Normalized Deep Gaussian Process Regression Monte Carlo Simulation

https://docs.gpytorch.ai/en/latest/examples/01_Exact_GPs/Simple_GP_Regression.html

https://docs.gpytorch.ai/en/stable/examples/06_PyTorch_NN_Integration_DKL/KISSGP_Deep_Kernel_Regression_CUDA.html

In [1]:
from core.AL import AL
from core.utils.plot_utils import results_plot, results_print

In [2]:
EXAMPLE = 'example1'

## Start structural reliability assessment and Active Learning procedure

In [ ]:
Results, History, Params, Data = AL(EXAMPLE)


Iteration 0
Hyperparameters: [1], SN: True, act_fun: ReLU
Early stopping at epoch 235
Best Loss: 3.4706976413726807 at epoch 133. Training loss: 2.128213882446289 and val. loss: 3.4706976413726807
  Training succeeded after 1 attempt(s).
avg loss = 3.4707
  New best model found at fold 1:
    layer sizes: [1], act fun: <class 'torch.nn.modules.activation.ReLU'>

Fold: 1, Avg. Loss: 3.4706976413726807

Early stopping at epoch 102
Best Loss: 1.3738690614700317 at epoch 0. Training loss: 3.82973051071167 and val. loss: 1.3738690614700317
  Training succeeded after 1 attempt(s).
avg loss = 1.3739
  New best model found at fold 2:
    layer sizes: [1], act fun: <class 'torch.nn.modules.activation.ReLU'>

Fold: 2, Avg. Loss: 1.3738690614700317

Early stopping at epoch 130
Best Loss: 2.168161392211914 at epoch 28. Training loss: 2.913984537124634 and val. loss: 2.168161392211914
  Training succeeded after 1 attempt(s).
avg loss = 2.1682
Fold: 3, Avg. Loss: 2.168161392211914

Early stopping a

## Display results and plot

In [ ]:
results_print(Results, History, Params)

In [ ]:
results_plot(History, Params)

# Sensitivity Analysis: Sobol' indices

Load necessary modules

In [ ]:
from core.utils.import_utils import load_core_modules, load_example_modules, load_surrogate_modules
from core.learning_function import evaluate_lf

_, learning_function, _ = load_core_modules(Params)
RVs, _, _ = load_example_modules(EXAMPLE)
predict, _ = load_surrogate_modules(Params)

Create a function to evaluate $\boldsymbol{x}$ with the trained surrogate

In [ ]:
def evaluate_model(x):
    if torch.cuda.is_available():
        x = x.cuda()
    # Evaluate the model
    preds = predict(Data.model, Data.likelihood, x)
    Y, _, _ = evaluate_lf(preds, learning_function)
    return Y.detach().cpu().numpy()

Perform Sobol' indices calculation

In [ ]:
# Import required libraries
import numpy as np
from SALib import ProblemSpec
from SALib.sample import saltelli
from SALib.analyze import sobol

# Define the problem (input parameters and their bounds)
problem = {
    'num_vars': len(RVs),  # Number of input parameters
    'names': [i['name'] for i in RVs],  # Names of the parameters
    'bounds': [[0, 1] for i in range(len(RVs))]  # Bounds for each parameter
}

# Generate samples using Saltelli's sampling method
param_values = saltelli.sample(problem, Params.sensitivity.n)

# # Define the model (replace this with your actual model)
# def evaluate_model(x):
#     # Example model: Ishigami function (a common test function for sensitivity analysis)
#     a = 7
#     b = 0.1
#     return np.sin(x[:, 0]) + a * np.sin(x[:, 1])**2 + b * x[:, 2]**4 * np.sin(x[:, 0])

# Evaluate the model at the sampled parameter values
Y = evaluate_model(torch.Tensor(param_values))

# Perform Sobol' sensitivity analysis
Si = sobol.analyze(problem, Y, print_to_console=True)

# Print the results
print("First-order Sobol' indices (S1):", Si['S1'])
print("Second-order Sobol' indices (S2):", Si['S2'])
print("Total-order Sobol' indices (ST):", Si['ST'])

Plot results

In [ ]:
# Define the LaTeX representations for your variables
# variable_names = [
#     r'Wind speed', r'$f_y g50$', r'$f_y g60$', r'$E$', r'$E_{guys}$', r'$A_{guys}$',
#     r'$f_{u_{guys}}$', r'Pre-tension', r'$w_{40}$', r'$w_{45}$', r'$w_{50}$', 
#     r'$w_{60}$', r'$w_{65}$', r'$w_{75}$', r'$t_{3mm}$', r'$t_{4mm}$', r'$t_{5mm}$', 
#     r'$t_{6mm}$', r'$\nu$', r'$d_c$', r'$d_{gw}$', r'$f_u g50$', r'$f_u g60$', r'$f_{ub}$', r'$A$',
#     r'$B$', r'$x$', r'$P$', r'$Q$', r'$R$'
# ]
variable_names = problem['names']

plt.rcParams.update({'font.size': 27})  # This will update the font size globally
plt.rcParams['figure.dpi'] = 300

# Assuming `Si` is your Sobol' indices, we use the first index (S1) for this plot example
# Replace `Si` with the appropriate object containing Sobol' indices
# Example data for Sobol' indices (you will replace this with the actual data from SALib)
# Si = {
#     'S1': np.random.rand(25),  # Sobol' first order indices for each of the 25 variables
#     'ST': np.random.rand(25)   # Sobol' total indices (optional)
# }

# Create the subplots with a 3x1 layout
fig, axes = plt.subplots(4, 1, figsize=(16, 36))  # 3 rows, 1 column, figsize for larger plots

# Plot your Sobol' first order indices (Si['S1']) on the first subplot
i = 0
# axes[i].bar(np.arange(len(Si['S1'])), Si['S1'])
axes[i].bar(np.arange(len(Si['S1'])), Si['S1'], yerr=Si['S1_conf'],
            capsize=5, color='skyblue', edgecolor='black', linewidth=1.5, error_kw={'linewidth': 2})
axes[i].set_yscale('log')

# Set x-axis tick positions and labels
axes[i].set_xticks(np.arange(len(variable_names)))
axes[i].set_xticklabels(variable_names, rotation=90, ha='center')

# Add titles and labels
axes[i].set_title('First Order Sobol\' Indices')
axes[i].set_xlabel('Random variables')
axes[i].set_ylabel('Sobol\' Indices (log scale)')


# Plot your Sobol' total order indices (Si['ST']) on the first subplot
i = 1
axes[i].bar(np.arange(len(Si['ST'])), Si['ST'], yerr=Si['ST_conf'],
            capsize=5, color='#D3D3D3', edgecolor='black', linewidth=1.5, error_kw={'linewidth': 2})
axes[i].set_yscale('log')

# Set x-axis tick positions and labels
axes[i].set_xticks(np.arange(len(variable_names)))
axes[i].set_xticklabels(variable_names, rotation=90, ha='center')

# Add titles and labels
axes[i].set_title('Total Order Sobol\' Indices')
axes[i].set_xlabel('Random variables')
axes[i].set_ylabel('Sobol\' Indices (log scale)')


# Plot your Sobol' first order indices (Si['S1']) on the first subplot
i = 2
# axes[i].bar(np.arange(len(Si['S1'])), Si['S1'])
axes[i].bar(np.arange(len(Si['S1'])), Si['S1'], yerr=Si['S1_conf'],
            capsize=5, color='skyblue', edgecolor='black', linewidth=1.5, error_kw={'linewidth': 2})
# axes[i].set_yscale('log')

# Set x-axis tick positions and labels
axes[i].set_xticks(np.arange(len(variable_names)))
axes[i].set_xticklabels(variable_names, rotation=90, ha='center')

# Add titles and labels
axes[i].set_title('First Order Sobol\' Indices')
axes[i].set_xlabel('Random variables')
axes[i].set_ylabel('Sobol\' Indices')


# Plot your Sobol' total order indices (Si['ST']) on the first subplot
i = 3
axes[i].bar(np.arange(len(Si['ST'])), Si['ST'], yerr=Si['ST_conf'],
            capsize=5, color='#D3D3D3', edgecolor='black', linewidth=1.5, error_kw={'linewidth': 2})
# axes[i].set_yscale('log')

# Set x-axis tick positions and labels
axes[i].set_xticks(np.arange(len(variable_names)))
axes[i].set_xticklabels(variable_names, rotation=90, ha='center')

# Add titles and labels
axes[i].set_title('Total Order Sobol\' Indices')
axes[i].set_xlabel('Random variables')
axes[i].set_ylabel('Sobol\' Indices')


# Adjust layout and spacing
fig.subplots_adjust(hspace=2)  # Adjust the vertical space between subplots

# Tight layout for final adjustments
plt.tight_layout()

# Show the figure
plt.show()

## Calculate $P_f$ with true performance function

In [ ]:
from core.utils.import_utils import load_reliability_modules, load_example_modules

Params = Params["Params"]
estimate_Pf, _ = load_reliability_modules(Params)
RVs, limit_state_function, _ = load_example_modules(EXAMPLE)

n = History['N_Init']
x = History['X']
x_candidate = History['MC_Sample']

# Retrieve original x_candidate
g1 = limit_state_function(x_candidate)
g2 = limit_state_function(x[n:, :])

Pf, _, _ = estimate_Pf(g1, g2, 0, Params['Params'])

N = g1.shape[0] + g2.shape[0]

print(f'Failure probability with {Params.reliability.method}: {Pf}')
print(f'CoV of Pf: {( Pf*(1-Pf)/N )**0.5 / Pf * 100:.2f}%')

## LOAD

In [ ]:
# from core.utils.serialization_utils import pickle_load
# pickle_load(EXAMPLE, ['Results', 'History', 'Params'])